In [ ]:
# import torchvision module to handle image manipulation
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



In [ ]:
test_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

FOR MLP

In [ ]:
train_target = np.zeros((60000,10))
validation_target = []
tmp = []
for i in range(len(train_set)):
  tmp.append(train_set[i][0].numpy().tolist())
  train_target[i][(train_set[i][1])] = 1
  validation_target.append(train_set[i][1])
train_target = np.array(train_target)
print(train_target)
# tmp = np.ndarray(tmp)
train_set = np.array(tmp)

[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


run this

In [ ]:
test_target = []
tmp2 = []
for i in test_set:
  tmp2.append(i[0].numpy().tolist())
  test_target.append(i[1])
test_target = np.array(test_target)
print(test_target)
test_set = np.array(tmp2)

[9 2 1 ... 8 1 5]


Don't have to run this Cross Validation part

In [ ]:
# train_mean = np.mean(train_set)
# train_set -= train_mean
# test_set -= train_mean
# train_set /= np.std(train_set, axis = 0)
# test_set /= np.std(test_set, axis = 0)

In [ ]:
# train_set = np.abs(train_set)
# train_set /= np.std(train_set, axis = 0)
# train_set

In [ ]:
# train_set_split = cross_validation_split_target(train_set)
# print(train_set_split)

In [ ]:
def normalize(z):
  z -= np.mean(z)
  z /= np.std(z)
  return z

In [ ]:
#ndarray, number of rows, dropout rate as input
def dropout(arr,numrows,rate):
  prob=np.random.uniform(size=arr[0].shape)
  tile=np.tile(prob, (numrows,1))
  drop=tile>rate #rate=0.n of the units will be 0
  copy_arr=drop*arr/(1-rate)
  return copy_arr


## 2 Layer

In [ ]:
import numpy as np
from scipy.special import logsumexp
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')
logistic = lambda z: 1./ (1 + np.exp(-z)) # Activation Function: sigmoid
relu = lambda z: np.maximum(z, 0)
tanh = lambda z: (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
leaky = lambda z: np.maximum(z,0) + 0.25*(np.minimum(z,0))

# def softmax(z):
#  print(f'z.shape[0]: {z.shape[0]}')

#  for i in range(z.shape[0]):
#   #  print(z[i])
#   # print(np.max(z[i]))
#   # z[i] -= np.max(np.absolute(z[i]))
#   sum = np.sum(np.exp(z[i]))
#   # z[i] -= np.max(z[i])   # Shift values down to prevent overflow
#   #  sum = np.sum(z[i])
#   #  print(sum)
#   #  print(z[i])
#   z[i] = np.exp(z[i])/sum
#   #  z[i] = z[i]/sum
#   #  for j in range(z.shape[1]):
#   #    z[i][j] = np.exp(j)/sum
#  return z

def softmax(z):
    # z -= np.max(z, axis=1, keepdims=True)
    numerator=np.exp(z)
    denominator=numerator.sum(1, keepdims=True)
    z=numerator/denominator
    return z

def crossEntropyLoss(x,y):
  tmp = np.zeros((y.shape[0],1))
  for i in range(x.shape[0]):
    tmp[i] = -1.0 * np.log(x[i][y[i]])
  return tmp

def softmaxEntropy(x, y, w, v):
  q = np.dot(x, v) #N x M
  z = logistic(q) #N x M
  u = np.dot(z, w) #N x C
  yh = softmax(u)
  nll = - np.mean(np.sum(u*y, 1) - logsumexp(u))
  return nll



class MLP:
    
    def __init__(self, M = 64): # num of hidden units
        self.M = M
            
    def fit(self, x, y, optimizer): # x is input, y is target i think, optimizer here is gradient descent
        N,D = x.shape
        def gradient(x, y, params):
            # copy_x=dropout(x, 50000,0.2)
            v, w = params
            q = np.dot(x, (v)) # N x M
            # print(f'q: {np.max(q)}')
            # tmp = np.max(q)
            # if tmp > 1e5:
            #   q /= tmp
            # print(f'q: {q}')
            z = tanh((q)) #N x M. first layer result | 60,000 x 128
            # z=dropout(z,50000,0.5)
            # z -= np.mean(z)
            # z /= np.std(z)
            # Check if np.dot(z,w) has any negative numbers, if it does, have to use relu first before softmax
            # THIS IS OUR PREDICTION second layer result
            # print(f'z: {z}')
            # print(f'z: {np.max(z)}')
            u = np.dot(z, w) # N x C
            # print(f'u: {np.max(u)}')
            yh = softmax(u)# N x C | 60,000 x 10
            # print(f'yh: {np.max(yh)}')
            # print(f'yh: {yh}')
            nll = - np.mean(np.sum(u*y, 1) - logsumexp(u)) # Cross entropy loss
            # print(f'nll: {nll}')
            # print(nll)
            # print(nll.shape)
            dy = yh - y #N x C     # THIS IS THE LOSS  = dL/dy
            # print(f'dy: {dy}')
            # dy = crossEntropyLoss(yh, y)
            # It uses L2 loss even tho it's classification. best loss function would be cross entropy loss
            dw = np.dot(z.T, dy)/N #M x C | 128 x 10
            # print(f'dw: {dw}')
            # dz = np.outer(dy, w) #N x M | 60,000 x 128
            dz = np.dot(dy, w.T) #N x M | 60,000 x 128
            # print(f'dz: {dz}')
            # x is 60,000 x 728 | x.T is 728 x 60,000
            dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
            # print(f'dv: {dv}')
            dparams = [dv, dw, nll]
            return dparams
        def predict2(x, v , w):
          q = np.dot(x, (v))
          z = tanh((q)) #N x M
          yh = softmax(np.dot(z, w))#N
          return yh
        
        w = np.random.randn(self.M,10) * .01
        v = np.random.randn(D,self.M) * .01
        # print(f'v: {np.sum(np.abs(v))}, w: {np.sum(np.abs(w))}')
        # print(w)
        # print(v)
        params0 = [v,w]
        self.params = optimizer.run(gradient, x, y, params0, predict2) # Optimizer returns the weights, the parameters
        return self
    
    def predict(self, x):
        v, w = self.params
        # print(f'v: {np.sum(np.abs(v))}, w: {np.sum(np.abs(w))}')
        # print(w)
        # print(v)
        q = np.dot(x, (v))
        # norm = np.linalg.norm(q)
        # q /= norm
        # tmp = np.max(q)
        # if tmp > 1e5:
        #   q /= tmp
        z = tanh((q)) #N x M
        # z -= np.mean(z)
        # z /= np.std(z)
        yh = softmax(np.dot(z, w))#N
        return yh

In [ ]:
from re import M
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8): # Epsilon is difference between the 2 weights
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        
    def run(self, gradient_fn, x, y, params, predict_fn):
        norms = np.array([np.inf])
        t = 1
        beta1 = 0.9
        beta2 = 0.999
        m0,m1,v0,v1 = 0,0,0,0
        while np.any(norms > self.epsilon) and t <= self.max_iters:
            # x2 = x[:]
            # y2 = y[:]
            # for i in range(3125):
            #   # mini_batch = x2[0]
            #   # print(mini_batch)
            #   # mini_target = y2[0]
            #   mini_batch = x2[:16]
            #   mini_target = y2[:16]
            #   x2 = x2[16:]
            #   # print(x2.shape)
            #   y2 = y2[16:]
            #   # print(y2.shape)
            #   # idx = np.random.randint(50000-i, size=1)
            #   # mini_batch = x[idx,:]
            #   # mini_target = y[idx,:]
            #   # x2 = np.delete(x2,idx,0)
            #   # y2 = np.delete(y2,idx,0)
            #   grad = gradient_fn(mini_batch, mini_target, params)
            #   # m0 = beta1 * m0 + (1 - beta1) * grad[0]
            #   # # print(f'm0:{m0}')
            #   # m1 = beta1 * m1 + (1 - beta1) * grad[1]
            #   # # print(f'm1:{m1}')
            #   # v0 = beta2 * v0 + (1 - beta2) * (grad[0]**2)
            #   # # print(f'v0:{v0}')
            #   # v1 = beta2 * v1 + (1 - beta2) * (grad[1]**2)
            #   # # print(f'v1:{v1}')
            #   # m0_corr = m0 / (1 - beta1**t)
            #   # # print(f'm0_corr:{m0_corr}')
            #   # m1_corr = m1 / (1 - beta1**t)
            #   # # print(f'm1_corr:{m1_corr}')
            #   # v0_corr = v0 / (1 - beta2**t)
            #   # # print(f'v0_corr:{v0_corr}')
            #   # v1_corr = v1 / (1 - beta2**t)
            #   # # print(f'v1_corr:{v1_corr}')
            #   # calc1 = (m0_corr/(np.sqrt(v0_corr)+self.epsilon))
            #   # # print(f'calc1:{calc1}')
            #   # calc2 = (m1_corr/(np.sqrt(v1_corr)+self.epsilon))
            #   # # print(f'calc2:{calc2}')
            #   # params[0] -= self.learning_rate * grad[0]*calc1
            #   # params[1] -= self.learning_rate * grad[1]*calc2
            #   params[0] -= self.learning_rate * grad[0]
            #   params[1] -= self.learning_rate * grad[1]
            grad = gradient_fn(x, y, params)
            if t % 5 == 0:
              # print(x.shape[0])
              # self.learning_rate *= 0.995
              bg = predict_fn(train[:], params[0], params[1])
              bg2 = np.mean(train_test[:]==np.argmax(bg,axis=1))
              yg = predict_fn(validation[:], params[0], params[1])
              yg2 = np.mean(validation_target[:]==np.argmax(yg,axis=1))
              gh = predict_fn(asdf2[:], params[0], params[1])
              gh2 = np.mean(test_target[:]==np.argmax(gh,axis=1))
              # bg = predict_fn(new_train_X[:1], params[0], params[1])
              # bg2 = np.mean(new_train_Y[:1]==np.argmax(bg,axis=1))
              # bg = predict_fn(train_X[:], params[0], params[1])
              # bg2 = np.mean(train_Y[:]==np.argmax(bg,axis=1))
              # yg = predict_fn(validate_X[:], params[0], params[1])
              # yg2 = np.mean(validate_Y[:]==np.argmax(yg,axis=1))
              # gh = predict_fn(test_X[:], params[0], params[1])
              # gh2 = np.mean(test_Y[:]==np.argmax(gh,axis=1))
              print(f'{t}, {grad[2]}, {bg2}, {yg2}, {gh2}, {self.learning_rate}')
            # if self.learning_rate > 0.15:
            #   self.learning_rate = self.learning_rate*0.995
            # print(f'v: {np.sum(np.abs(params[0]))}, w: {np.sum(np.abs(params[1]))}')
            params[0] -= self.learning_rate * grad[0]
            params[1] -= self.learning_rate * grad[1]
            t += 1
            norms = np.array([np.linalg.norm(grad[g]) for g in range(2)])
            # print(norms)
        return params

In [ ]:
asdf = np.reshape(train_set, (60000,784))
asdf2 = np.reshape(test_set, (10000,784))
train = asdf[:50000]
train_test = validation_target[:50000]
validation_target = validation_target[50000:]
validation = asdf[50000:]
train_target = train_target[:50000]
# asdf3 = np.reshape(train_target, (32,1875))
asdf.shape
train_target.shape
asdf2.shape
test_set.shape

(10000, 1, 28, 28)

In [ ]:
len(train_test)

10000

From here is just Sklearn, dont have to run

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X, y = make_classification(n_samples=100, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=1)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes = (128,2),random_state=1, max_iter=250).fit(asdf[:50000], train_target[:])

In [ ]:
a = clf.predict(asdf2)

In [ ]:
np.mean(a == test_target)

0.0

From here is testing, please run

In [ ]:
x = train[:]
y = train_target[:]
model = MLP(M=128)
optimizer = GradientDescent(learning_rate=0.53, max_iters=250)
yh = model.fit(x, y, optimizer)

5, 13.057589378585389, 0.4681, 0.4734, 0.4634, 0.53
10, 12.911906355147364, 0.38196, 0.3867, 0.3749, 0.53
15, 12.629008158515663, 0.5069, 0.5212, 0.5026, 0.53
20, 12.38298527195807, 0.59612, 0.6083, 0.5895, 0.53
25, 12.236422956855042, 0.62076, 0.6283, 0.6146, 0.53
30, 12.146592538609799, 0.63936, 0.6461, 0.6317, 0.53
35, 12.091176424009353, 0.65358, 0.66, 0.645, 0.53
40, 12.060177250787918, 0.66504, 0.6733, 0.6567, 0.53
45, 12.047313328419268, 0.67712, 0.6851, 0.6683, 0.53
50, 12.047117785416683, 0.68634, 0.692, 0.6755, 0.53
55, 12.261886945103498, 0.62606, 0.6265, 0.615, 0.53
60, 12.142516088125447, 0.65128, 0.6462, 0.6405, 0.53
65, 12.40089793049705, 0.70178, 0.7059, 0.6911, 0.53
70, 12.231649227381881, 0.69444, 0.6919, 0.6837, 0.53
75, 12.31669008479125, 0.68668, 0.6859, 0.6837, 0.53
80, 12.362174362563433, 0.70402, 0.6992, 0.6968, 0.53
85, 12.331465367645595, 0.70598, 0.7078, 0.7028, 0.53
90, 12.410493719668628, 0.71132, 0.7086, 0.7053, 0.53
95, 12.393892712719357, 0.72114, 0.7226

In [ ]:
yg = yh.predict(asdf2[:])
np.mean(test_target==np.argmax(yg,axis=1))

0.1

## No Hidden Layer

In [ ]:
import numpy as np
from scipy.special import logsumexp
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')
logistic = lambda z: 1./ (1 + np.exp(-z)) # Activation Function: sigmoid
relu = lambda z: np.maximum(z, 0)
tanh = lambda z: (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
leaky = lambda z: np.maximum(z,0) + 0.25*(np.minimum(z,0))

def softmax(z):
    # z -= np.max(z, axis=1, keepdims=True)
    numerator=np.exp(z)
    denominator=numerator.sum(1, keepdims=True)
    z=numerator/denominator
    return z

class MLP_0:
    
    def __init__(self, M = 64): # num of hidden units
        self.M = M
            
    def fit(self, x, y, optimizer): # x is input, y is target i think, optimizer here is gradient descent
        N,D = x.shape
        transform = np.full((N,10),1) # Set all to 1, every class with have same probability
        # transform = np.random.randn(N,10) * .01 # Set random variables
        self.v = np.dot(x.T,transform)
        return self
    
    def predict(self, x):
        yh = softmax(np.dot(x,self.v)) #N
        return yh

## 1 Hidden Layer

In [2]:
import numpy as np
from scipy.special import logsumexp
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')
logistic = lambda z: 1./ (1 + np.exp(-z)) # Activation Function: sigmoid
relu = lambda z: np.maximum(z, 0)
tanh = lambda z: (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
leaky = lambda z: np.maximum(z,0) + 0.25*(np.minimum(z,0))

def softmax(z):
    # z -= np.max(z, axis=1, keepdims=True)
    numerator=np.exp(z)
    denominator=numerator.sum(1, keepdims=True)
    z=numerator/denominator
    return z

# def crossEntropyLoss(x,y):
#   tmp = np.zeros((y.shape[0],1))
#   for i in range(x.shape[0]):
#     tmp[i] = -1.0 * np.log(x[i][y[i]])
#   return -np.sum(tmp)

def softmaxEntropy(x, y, w, v):
  q = np.dot(x, v) #N x M
  z = logistic(q) #N x M
  u = np.dot(z, w) #N x C
  yh = softmax(u)
  nll = - np.mean(np.sum(u*y, 1) - logsumexp(u))
  return nll



class MLP_1:
    
    def __init__(self, M = 128): # num of hidden units
        self.M = M
            
    def fit(self, x, y, optimizer): # x is input, y is target i think, optimizer here is gradient descent
        N,D = x.shape
        def gradient(x, y, params):
            #q_target = np.dot
            v,w = params
            #q = np.dot(x, v) # N x M
            z = relu(normalize(x))
            #zz=np.reshape(z, (306250,128))
            
            #z = logistic((q)) #N x M. first layer result | 60,000 x 128
            u = np.dot(z,w) # N x C
            yh = softmax(u)# N x C | 60,000 x 10
            # nll = softmaxEntropy(x,y,w,v)
            dy = yh - y #N x C     # THIS IS THE LOSS  = dL/dy
            dw = np.dot(z.T, dy)/N #M x C | 128 x 10
            dz = np.dot(dy, w.T) #N x M | 60,000 x 128
            #dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
            dparams = [v, dw]
            return dparams
        def predict2(x, params):
          v,w=params
          q = np.dot(x, v)
          #z = logistic((q)) #N x M
          z=relu(normalize(x))
          #zz=np.reshape(z,(50000,128))
          yh = softmax(np.dot(z, w))#N
          return yh
        
        # self.v = np.random.randn(D,self.M) * .01
        v = np.full((D,self.M),1) #stays the same does not change
        #v=np.random.randn(D,self.M) *.01 #D=784, N=50000 M=128
        # self.w = np.full((self.M,10),1)
        w = np.random.randn(self.M,10) * .01 #this is adjusted
        #w=np.full((self.M,10),1)
        params0 = [v,w]
        self.params = optimizer.run(gradient, x, y, params0, predict2)
        #self.w = optimizer.run(gradient, x, y, self.w, predict2) # Optimizer returns the weights, the parameters
        return self
    
    def predict(self, x):
        v,w=self.params
        q = np.dot(x, v)
        #z = logistic((q)) #N x M
        z=relu(normalize(x))
        #zz=np.reshape(z,(50000,128))
        yh = softmax(np.dot(z, w))#N
        return yh

## 1 Hidden Layer Gradient Descent

In [ ]:
from re import M
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8): # Epsilon is difference between the 2 weights
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        
    def run(self, gradient_fn, x, y, params, predict_fn):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t <= self.max_iters:
            grad = gradient_fn(x, y, params)
            if t % 1 == 0:
              bg = predict_fn(train[:], params)
              bg2 = np.mean(train_test[:]==np.argmax(bg,axis=1))
              yg = predict_fn(validation[:], params)
              yg2 = np.mean(validation_target[:]==np.argmax(yg,axis=1))
              gh = predict_fn(asdf2[:], params)
              gh2 = np.mean(test_target[:]==np.argmax(gh,axis=1))
              print(f'{t}, {bg2}, {yg2}, {gh2}, {self.learning_rate}')

              #print(f'{t}, {gh2}, {self.learning_rate}')
            # if t < 1*self.max_iters:
            #   self.learning_rate = self.learning_rate*0.99
            params[1] -= self.learning_rate * grad[1]
            t += 1
            norms = np.array([np.linalg.norm(grad[1])])
        return params

In [ ]:
x = train[:]
y = train_target[:]
model = MLP_1(M=128)
optimizer = GradientDescent(learning_rate=0.05, max_iters=500)
yh = model.fit(x, y, optimizer)

1, 0.07576, 0.0746, 0.0802, 0.05
2, 0.35582, 0.3539, 0.3543, 0.05
3, 0.55176, 0.5541, 0.5476, 0.05
4, 0.55144, 0.554, 0.5413, 0.05
5, 0.60162, 0.6024, 0.5954, 0.05
6, 0.59218, 0.5949, 0.5843, 0.05
7, 0.60952, 0.6104, 0.6024, 0.05
8, 0.60328, 0.6081, 0.5968, 0.05
9, 0.61328, 0.6129, 0.6059, 0.05
10, 0.61016, 0.6151, 0.6033, 0.05
11, 0.61714, 0.6159, 0.6094, 0.05
12, 0.61542, 0.6206, 0.6093, 0.05
13, 0.62132, 0.6203, 0.6148, 0.05
14, 0.62178, 0.6252, 0.6157, 0.05
15, 0.62602, 0.6241, 0.6202, 0.05
16, 0.6277, 0.632, 0.6227, 0.05
17, 0.63078, 0.6289, 0.6269, 0.05
18, 0.6351, 0.6402, 0.6299, 0.05
19, 0.63588, 0.6351, 0.6318, 0.05
20, 0.64258, 0.6457, 0.6368, 0.05
21, 0.6411, 0.6406, 0.6366, 0.05
22, 0.65018, 0.6522, 0.6431, 0.05
23, 0.64784, 0.6457, 0.6423, 0.05
24, 0.65748, 0.6605, 0.6498, 0.05
25, 0.65478, 0.6529, 0.648, 0.05
26, 0.66462, 0.6679, 0.6582, 0.05
27, 0.66112, 0.6593, 0.6551, 0.05
28, 0.67136, 0.6753, 0.6657, 0.05
29, 0.667, 0.6652, 0.6588, 0.05
30, 0.67764, 0.6802, 0.6702, 0.

In [ ]:
x = train[:]
y = train_target[:]
model = MLP_0(M=128)
yh = model.fit(x, y, optimizer)

In [ ]:
yg = yh.predict(asdf2[:])

In [ ]:
yg[:5]

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [ ]:
yg.shape

(10000, 10)

ACCURACY:

In [ ]:
yg = yh.predict(asdf2[:])
np.mean(test_target==np.argmax(yg,axis=1))

0.7912

In [ ]:
from keras.datasets import fashion_mnist
(train_X,train_Y), (test_X,test_Y) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_Y

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
train_X = np.reshape(train_X, (60000,784))
validate_X = train_X[:10000]
validate_Y = train_Y[:10000]
train_Y = train_Y[:50000]
test_X = np.reshape(test_X, (10000,784))

In [ ]:
train_Y_fixed = np.zeros((50000,10))
for i in range(len(train_Y)):
  train_Y_fixed[i][train_Y[i]] = 1

In [ ]:
train_Y_fixed.shape

(50000, 10)

In [ ]:
train_X = train_X[:50000]

In [ ]:
x = train_X[:]
y = train_Y_fixed[:]
model = MLP(M=128)
optimizer = GradientDescent(learning_rate=1.2, max_iters=250)
yh = model.fit(x, y, optimizer)

5, 13.12254919649972, 0.10036, 0.0969, 0.0996, 1.2
10, 13.12244736963049, 0.10036, 0.0969, 0.0996, 1.2
15, 13.122415733344708, 0.10036, 0.0969, 0.0996, 1.2
20, 13.122404699509032, 0.10036, 0.0969, 0.0996, 1.2
25, 13.122398193943642, 0.10036, 0.0969, 0.0996, 1.2


KeyboardInterrupt: ignored

In [ ]:
indexes =[]
classes = [0,1,2,3,4,5,6,7,8,9]
for clas in classes:
  count = 0 
  for i in range(len(train_Y)):
    if clas == train_Y[i]:
      indexes.append(i)
      count += 1
      if  count == 1:
        break

new_train_Y = [train_Y[i] for i in indexes] 
new_train_X = [train_X[i] for i in indexes]
new_train_Y=np.array(new_train_Y)
new_train_X=np.array(new_train_X)
new_train_Y_fixed = np.zeros((10,10))
for i in range(len(new_train_Y)):
  new_train_Y_fixed[i][new_train_Y[i]] = 1

In [ ]:
new_train_Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=uint8)

In [ ]:
new_train_Y.shape
# new_train_Y[:10]

(10000,)

In [ ]:
x = new_train_X[:1]
y = new_train_Y_fixed[:1]
model = MLP(M=128)
optimizer = GradientDescent(learning_rate=0.00005, max_iters=250)
yh = model.fit(x, y, optimizer)

5: 2.2505445491336173, 0.0, 0.1025, 0.0971, lr: 5e-05
10: 2.2280455697006634, 0.0, 0.1023, 0.0967, lr: 5e-05
15: 2.2131213531496563, 0.0, 0.1026, 0.0972, lr: 5e-05
20: 2.1743226070647106, 1.0, 0.1038, 0.0999, lr: 5e-05
25: 2.1616711556779458, 1.0, 0.1085, 0.103, lr: 5e-05
30: 2.149703551417788, 1.0, 0.1113, 0.1087, lr: 5e-05
35: 2.1378829869219995, 1.0, 0.1157, 0.1157, lr: 5e-05
40: 2.1261441481732035, 1.0, 0.1189, 0.1208, lr: 5e-05
45: 2.1144641227895296, 1.0, 0.123, 0.124, lr: 5e-05
50: 2.102827923523081, 1.0, 0.1276, 0.1296, lr: 5e-05
55: 2.091215126935192, 1.0, 0.13, 0.1327, lr: 5e-05
60: 2.079576735554876, 1.0, 0.1321, 0.1341, lr: 5e-05
65: 2.0677967077566715, 1.0, 0.1303, 0.1363, lr: 5e-05
70: 2.0559327443261997, 1.0, 0.1286, 0.1355, lr: 5e-05
75: 2.044219710926187, 1.0, 0.127, 0.1322, lr: 5e-05
80: 2.0326005458800256, 1.0, 0.1246, 0.13, lr: 5e-05
85: 2.021022748584861, 1.0, 0.122, 0.1263, lr: 5e-05
90: 2.0094485226829724, 1.0, 0.1174, 0.1212, lr: 5e-05
95: 1.9978052895264984, 1.